### Model Account Sample

In [1]:
from absbox import Generic,API,EnginePath,readAccsCf

accountSample = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":25
          ,"status":"current"}]]}
    ,None
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30},"feeStart":"2021-06-15"}),)
    ,{"amortizing":[
         # release excess cash from reserve account
         ["transfer","reserveAcc","acc01",{"reserve":"excess"}]
         # pay fee & interest
         ,["payFee","acc01",['trusteeFee'],{"support":["account","reserveAcc"]}]
         ,["accrueAndPayInt","acc01",["A1"],{"support":["account","reserveAcc"]}]
         # release all cash from reserve if A1 is paid off
         # otherwise, fill up reserve account 
         ,["IfElse",[("isPaidOff","A1"),True]
                 ,[["transfer","reserveAcc","acc01"]]
                 ,[["transfer","acc01","reserveAcc",{"reserve":"gap"}]]
          ]
         # pay principal to A1 & B
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

localAPI = API(EnginePath.DEV,check=False)

Connecting engine server -> https://absbox.org/api/dev

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅Connected, local lib:0.46.5, server:0.50.1

#### Waterfall to run with

In [2]:
accountSample.waterfall['amortizing']

[['transfer', 'reserveAcc', 'acc01', {'reserve': 'excess'}],
 ['payFee', 'acc01', ['trusteeFee'], {'support': ['account', 'reserveAcc']}],
 ['accrueAndPayInt', 'acc01', ['A1'], {'support': ['account', 'reserveAcc']}],
 ['IfElse',
  [('isPaidOff', 'A1'), True],
  [['transfer', 'reserveAcc', 'acc01']],
  [['transfer', 'acc01', 'reserveAcc', {'reserve': 'gap'}]]],
 ['payPrin', 'acc01', ['A1']],
 ['payPrin', 'acc01', ['B']],
 ['payIntResidual', 'acc01', 'B']]

#### with fixed target amount

In [3]:
accountSample.accounts = (("acc01",{"balance":0}) # distribution account
                          ,("reserveAcc",{"balance":100
                                          ,"type":("fix",100)})) # reserve account

r = localAPI.run(accountSample
               ,poolAssump = ("Pool",("Mortgage",None,None,None,None)
                                       ,None
                                       ,None)
               ,runAssump = None
               ,read=True)

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
# no cash collected at 06-30
r['accounts']['acc01'].loc["2021-06-30"]

,balance,change,memo
date,,,
2021-06-30,0.0,0.0,<Pool:CollectedInterest>
2021-06-30,0.0,0.0,<Pool:CollectedPrincipal>
2021-06-30,0.0,0.0,<Pool:CollectedPrepayment>
2021-06-30,0.0,0.0,<Pool:CollectedRecoveries>


#### Insufficent cash from distribution acccount

In [5]:
r['accounts']['acc01'].loc["2021-07-26"]

,balance,change,memo
date,,,
2021-07-26,0.0,0.0,"<Transfer:reserveAcc,acc01>"
2021-07-26,0.0,0.0,<SeqPayFee:trusteeFee>
2021-07-26,0.0,0.0,<PayInt:A1>
2021-07-26,0.0,0.0,"<Transfer:acc01,reserveAcc>"
2021-07-26,0.0,0.0,<PayPrin:A1>
2021-07-26,0.0,0.0,<PayPrin:B>
2021-07-26,0.0,0.0,<PayYield:B>


#### Reserve account is helping out

In [6]:
r['accounts']['reserveAcc'].loc["2021-07-26"]

,balance,change,memo
date,,,
2021-07-26,100.00,0.00,"<Transfer:reserveAcc,acc01>"
2021-07-26,70.00,-30.00,<SupportDraw:>
2021-07-26,62.14,-7.86,<SupportDraw:>
2021-07-26,62.14,0.00,"<Transfer:acc01,reserveAcc>"


#### A1 is being paid interest

In [7]:
r['bonds']['A1'].loc["2021-07-26":"2021-08-26"]

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2021-07-26,1000.0,7.86,0.0,0.07,7.86,0,0,1.0,<PayInt:A1>
2021-08-20,1000.0,4.79,0.0,0.07,4.79,0,0,1.0,<PayInt:A1>


#### fee is paid

In [8]:
r['fees']['trusteeFee'].loc["2021-07-26":"2021-08-26"]

,balance,payment,due
date,,,
2021-07-26,0,30,0
2021-08-20,0,0,0


#### Reserve Account is being fill up

In [9]:
readAccsCf(r['accounts']).loc["2021-08-20":"2021-10-20"]

acc01         reserveAcc        
                  balance  change    balance  change
date       daySeq                                   
2021-08-20 0         0.00    0.00      62.14    0.00
           1         0.00    0.00      62.14    0.00
           2         0.00    0.00      57.35   -4.79
           3         0.00    0.00      57.35    0.00
           4         0.00    0.00        NaN     NaN
           5         0.00    0.00        NaN     NaN
           6         0.00    0.00        NaN     NaN
2021-08-31 0        14.66   14.66        NaN     NaN
           1        95.82   81.16        NaN     NaN
           2        95.82    0.00        NaN     NaN
           3        95.82    0.00        NaN     NaN
2021-09-20 0       153.17   57.35       0.00  -57.35
           1       153.17    0.00       0.00    0.00
           2       147.23   -5.94       0.00    0.00
           3        47.23 -100.00     100.00  100.00
           4         0.00  -47.23        NaN     NaN
           5         0.00    0.00        NaN     NaN
           6         0.00    0.00        NaN     NaN
2021-09-30 0        14.12   14.12        NaN     NaN
           1        95.82   81.70        NaN     NaN
           2        95.82    0.00        NaN     NaN
           3        95.82    0.00        NaN     NaN
2021-10-20 0       191.64   95.82       4.18  -95.82
           1       191.64    0.00       4.18    0.00
           2       186.16   -5.48       4.18    0.00
           3        90.34  -95.82     100.00   95.82
           4         0.00  -90.34        NaN     NaN
           5         0.00    0.00        NaN     NaN
           6         0.00    0.00        NaN     NaN

#### When A1 is being paid off

In [178]:
r['bonds']['A1'].loc["2022-07-20":"2022-08-20"]

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2022-07-20,25.92,0.69,95.14,0.07,95.83,0,0,0.02592,"[<PayInt:A1>, <PayPrin:A1>]"
2022-08-20,0.00,0.15,25.92,0.07,26.07,0,0,0.00000,"[<PayInt:A1>, <PayPrin:A1>]"


#### Reserve account is releasing

In [179]:
r['accounts']['acc01'].loc["2022-09-20"]

,balance,change,memo
date,,,
2022-09-20,95.83,0.00,"<TransferBy:reserveAcc,acc01,TillSource>"
2022-09-20,95.83,0.00,<SeqPayFee:trusteeFee>
2022-09-20,95.83,0.00,<PayInt:A1>
2022-09-20,195.83,100.00,"<Transfer:reserveAcc,acc01>"
2022-09-20,195.83,0.00,<PayPrin:A1>
2022-09-20,0.00,-195.83,<PayPrin:B>
2022-09-20,0.00,0.00,<PayYield:B>


#### With a formula

In [10]:
accountSample.accounts = (("acc01",{"balance":0})
                          ,("reserveAcc",{"balance":100
                                          ,"type":("target"
                                                   ,("*",("poolBalance",),0.02)  #<Formula> 
                                                  )}))

r2 = localAPI.run(accountSample
               ,poolAssump = ("Pool",("Mortgage",None,None,None,None)
                                       ,None
                                       ,None)
               ,runAssump = None
               ,read=True)

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [11]:
readAccsCf(r2['accounts']).loc[:"2022-10-20"]

acc01        reserveAcc       
                  balance change    balance change
date       daySeq                                 
2021-06-15 0         0.00   0.00        NaN    NaN
           1         0.00   0.00        NaN    NaN
           2         0.00   0.00        NaN    NaN
           3         0.00   0.00        NaN    NaN
2021-06-30 0         0.00   0.00        NaN    NaN
...                   ...    ...        ...    ...
2022-10-20 2        95.82   0.00        0.0    0.0
           3        95.82   0.00        0.0    0.0
           4        95.82   0.00        NaN    NaN
           5         0.00 -95.82        NaN    NaN
           6         0.00   0.00        NaN    NaN

[180 rows x 4 columns]

#### distributioin account is collecting excess cash from reserve account

In [12]:
r2['accounts']['acc01'].loc['2021-11-20']

,balance,change,memo
date,,,
2021-11-20,136.56,40.74,"<Transfer:reserveAcc,acc01>"
2021-11-20,136.56,0.00,<SeqPayFee:trusteeFee>
2021-11-20,131.79,-4.77,<PayInt:A1>
2021-11-20,92.70,-39.09,"<Transfer:acc01,reserveAcc>"
2021-11-20,0.00,-92.70,<PayPrin:A1>
2021-11-20,0.00,0.00,<PayPrin:B>
2021-11-20,0.00,0.00,<PayYield:B>


#### With reinvestment setup

#### fix rate

In [13]:
accountSample.accounts = (("acc01",{"balance":0
                                    ,"interest":{"period":"QuarterEnd"  # <DatePattern>
                                                ,"rate":0.05
                                                ,"lastSettleDate":"2022-11-02"} })
                          ,("reserveAcc",{"balance":100
                                          ,"type":("target",("*",("poolBalance",),0.02))}))

r3 = localAPI.run(accountSample
               ,poolAssump = ("Pool",("Mortgage",None,None,None,None)
                                       ,None
                                       ,None)
               ,runAssump = None
               ,read=True)

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [202]:
r3['accounts']['acc01'].loc["2023-06-30"]

,balance,change,memo
date,,,
2023-06-30,0.80,0.80,<BankInterest:>
2023-06-30,0.80,0.00,<Pool:CollectedRecoveries>
2023-06-30,0.80,0.00,<Pool:CollectedPrepayment>
2023-06-30,94.74,93.94,<Pool:CollectedPrincipal>
2023-06-30,96.63,1.89,<Pool:CollectedInterest>


#### float rate

In [15]:
accountSample.accounts = (("acc01",{"balance":0})
                          ,("reserveAcc",{"balance":100
                                          ,"type":("target",("*",("poolBalance",),0.02))
                                          ,"interest":{"period":"YearEnd"
                                                       ,"reset":"QuarterEnd"
                                                       ,"index":"SOFR3M"
                                                       ,"spread":0.02
                                                       ,"rate":0.03
                                                       ,"lastSettleDate":"2020-11-02"}
                                         }))

r4 = localAPI.run(accountSample
               ,poolAssump = ("Pool",("Mortgage",None,None,None,None)
                                       ,None
                                       ,None)
               ,runAssump = [("interest",("SOFR3M",0.04))]
               ,read=True)

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [16]:
r4['accounts']['reserveAcc'].loc["2021-12-31"]

,balance,change,memo
date,,,
2021-12-31,42.19,4.75,<BankInterest:>
2021-12-31,42.19,0.00,<BankInterest:>
